# Import librairie #

In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [2]:
# Charger le .env depuis la racine du projet
from pathlib import Path
dotenv_path = r'C:\Penny\.env'
load_result = load_dotenv(dotenv_path=dotenv_path)
print(f"Chargement .env : {'[OK] Reussi' if load_result else '[ERREUR] Echec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [3]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [4]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table Bank Pennylane ##

In [5]:
# Tables bank accounts Pennylane
tables_extended = [
    'bank_accounts'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\1637964975.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ bank_accounts (18 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
   table_name  columns                                                           column_list
bank_accounts       18 id, company_id, company_name, establishment_id, establishment_name...


## Liste colonnes Bank Pennylane ##

In [6]:
# Afficher les colonnes bank accounts
tables_found = [
    'bank_accounts'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.bank_accounts
Colonnes (18):
   1. id
   2. company_id
   3. company_name
   4. establishment_id
   5. establishment_name
   6. name
   7. connection
   8. accounting_plan_item_id
   9. accounting_plan_item_number
  10. accounting_plan_item_label
  11. currency
  12. is_visible
  13. is_synchronized
  14. balance
  15. last_sync_success
  16. last_sync_at
  17. created_at
  18. updated_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\785808616.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données bank ##

In [7]:
# Charger le bank
query_gl = "SELECT * FROM pennylane.bank_accounts;"
df_bank_accounts = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_bank_accounts)} lignes, {len(df_bank_accounts.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_67984\3980678004.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bank_accounts = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 2 lignes, 18 colonnes


## Affichage des colonnes disponibles ##

In [8]:
print("Colonnes disponibles :")
for i, col in enumerate(df_bank_accounts.columns, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. id
   2. company_id
   3. company_name
   4. establishment_id
   5. establishment_name
   6. name
   7. connection
   8. accounting_plan_item_id
   9. accounting_plan_item_number
  10. accounting_plan_item_label
  11. currency
  12. is_visible
  13. is_synchronized
  14. balance
  15. last_sync_success
  16. last_sync_at
  17. created_at
  18. updated_at


## Suppresion de colonnes ##

In [9]:
# Colonnes à supprimer
columns_to_drop = ["id", "company_id","establishment_id","created_at", "updated_at"]

# Vérifier quelles colonnes existent réellement
existing_cols = [col for col in columns_to_drop if col in df_bank_accounts.columns]
missing_cols = [col for col in columns_to_drop if col not in df_bank_accounts.columns]

print(f"Colonnes supprimées : {existing_cols}")
if missing_cols:
    print(f"Colonnes inexistantes (ignorées) : {missing_cols}")

# Supprimer les colonnes
df_bank_accounts = df_bank_accounts.drop(columns=existing_cols)

print(f"\n✓ Résultat : {len(df_bank_accounts.columns)} colonnes restantes")
print(f"Colonnes finales :\n{df_bank_accounts.columns.tolist()}")

Colonnes supprimées : ['id', 'company_id', 'establishment_id', 'created_at', 'updated_at']

✓ Résultat : 13 colonnes restantes
Colonnes finales :
['company_name', 'establishment_name', 'name', 'connection', 'accounting_plan_item_id', 'accounting_plan_item_number', 'accounting_plan_item_label', 'currency', 'is_visible', 'is_synchronized', 'balance', 'last_sync_success', 'last_sync_at']


## Affichage type clonne ##

In [10]:
print(f"Types de données des {len(df_bank_accounts.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_bank_accounts.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 13 colonnes :

   1. company_name                   → object
   2. establishment_name             → object
   3. name                           → object
   4. connection                     → object
   5. accounting_plan_item_id        → int64
   6. accounting_plan_item_number    → object
   7. accounting_plan_item_label     → object
   8. currency                       → object
   9. is_visible                     → bool
  10. is_synchronized                → bool
  11. balance                        → float64
  12. last_sync_success              → bool
  13. last_sync_at                   → datetime64[ns]


## Conversion des types de colonnes ##

In [11]:
# Conversion des types
df_bank_accounts = df_bank_accounts.astype({
    'balance': 'float64'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_bank_accounts.columns if col not in ['balance']]
df_bank_accounts[columns_to_string] = df_bank_accounts[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_bank_accounts.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. company_name                   → object
   2. establishment_name             → object
   3. name                           → object
   4. connection                     → object
   5. accounting_plan_item_id        → object
   6. accounting_plan_item_number    → object
   7. accounting_plan_item_label     → object
   8. currency                       → object
   9. is_visible                     → object
  10. is_synchronized                → object
  11. balance                        → float64
  12. last_sync_success              → object
  13. last_sync_at                   → object


## Connexion PostgreSQL local ##

In [12]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [13]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##

## Export vers PostgreSQL local ##

In [ ]:
# Export DataFrame vers PostgreSQL local
df_bank_accounts.to_sql(
    name='bank_accounts',
    con=engine,
    schema='pennylane',
    if_exists='replace',
    index=False,
    method='multi',
    chunksize=1000
)

print(f"[OK] Table 'bank_accounts' exportee : {len(df_bank_accounts)} lignes")

# Fermer connexions
conn.close()
conn_pg.close()

print("[OK] Connexions fermees")